# Import

In [1]:
import dlib
import numpy as np
import pandas as df
import cv2
import sys
import ffmpeg
import os
import natsort
import glob
import csv

ModuleNotFoundError: No module named 'dlib'

# Rotation management
In some situations opencv reads a video but cannot understand the rotation codecs, and rotates the video and the extracted frames, this method is used to read the codecs and rotate the video in the right way, because dlib does not recognize the rotated faces

In [ ]:

def check_rotation(path_video_file):
     # returns the meta-data of the dictionary-style videos
    meta_dict = ffmpeg.probe(path_video_file)

    # rotate parameter search
    rotateCode = None
    try:
        if int(meta_dict['streams'][0]['tags']['rotate']) == 90: #see if in the video meta-data the roate parameter is 90
            rotateCode = cv2.ROTATE_90_CLOCKWISE
        elif int(meta_dict['streams'][0]['tags']['rotate']) == 180: #see if in the video meta data the roate parameter is 180
            rotateCode = cv2.ROTATE_180
        elif int(meta_dict['streams'][0]['tags']['rotate']) == 270: #see if in the video meta data the roate parameter is 270
            rotateCode = cv2.ROTATE_90_COUNTERCLOCKWISE
    #exception if it doesn't find the rotate parameter
    except:
        print('non trova attributo rotate')

    return rotateCode

# Method to return the correct rotation

In [ ]:
def correct_rotation(frame, rotateCode):  
    return cv2.rotate(frame, rotateCode)

# Method that provides the Images array
this method takes the extracted frames in the video that have been saved in the folder folder, and loads them into the images array in sequential order

In [ ]:
def load_images_from_folder(folder):
    
    images = []
    dirFiles = os.listdir(folder)

    dirFiles = natsort.natsorted(dirFiles,reverse=False)#sorts the image paths sequentially
    i = 0
    print('array population in progress...')
    for filename in dirFiles:
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    print('population finished')
    
    return images


# Method for calculating the confidence score
Calculate confidence for detection face

In [2]:
def calcoloConfidenceScore(detector, image, faces, idImage):
    dets, scores, idx = detector.run(image, 1, -1)
    for i, d in enumerate(faces):
        print("Detection {}, score: {}, face_type:{}, idImage:{}".format(#in score vi sarà la confidence del volto rilevato
            d, scores[i], idx[i], idImage))
    return scores

# Method for face detection
dlib is applied for face detection and a csv is written in which 0 if the face is present and 1 if not present is associated to each frame

In [ ]:
def detect_face(image, i ,video):
    originalImage = image.copy()
    best_face=0
    max_area=-1
    outputFolderDetected = 'face_detected/'
    outputFolderNotDetected = 'face_not_detected/'
    #detector inizialization
    detector = dlib.get_frontal_face_detector()
    name_frame=vid+"_"+"{0:0=12d}".format(i);
    #faces detected by the detector
    faces = detector(image)
    with open( 'dlib.csv', mode='a', newline='') as f:
        writer = csv.writer(f)
       
        #ConfidenceScore calculation
        if(len(faces)>0):
            score = calcoloConfidenceScore(detector, originalImage, faces, i)
            x=0
            cord = []
            best_face = 0
            for face in faces:
                print(face)
                x1 = face.left()#first parameter
                y1 = face.top()#second parameter
                x2 = face.right()#third parameter
                y2 = face.bottom()#fourth parameter
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 3)
                height = y1 - y2
                width = x2 - x1
                area = height * width
                if(score[x]> 0.1 and score[x]> best_face):
                    cord.insert(0,x1)
                    cord.insert(1,y1)
                    cord.insert(2,x2)
                    cord.insert(3,y2)
                    best_face = score[x]
                    print(cord[0])
                x+=1
                if(best_face>0):
                    writer.writerow([str(name_frame)]+['1']+ [cord[0]]+ [cord[1]]+ [cord[2]]+ [cord[3]])
                else:
                    writer.writerow([str(name_frame)]+['0'])
        else:
            print('no face was detected')
            writer.writerow([str(name_frame)]+['0'])
        


# Face detection and csv writing
In this portion of code, the methods for populating the array with the frames of the various videos are called, after which dlib is applied for the detection of faces through the call to the "detect_face" method.

In [ ]:
types = ('*.mp4', '*.MOV') # the tuple of file types
file_source = []
for files in types:
    file_source.extend(glob.glob(files))

folderFrameOutput = 'prova3/'
result_directory='directoryCsv/'

# input a video whose frames you want to extract
for inputVideo in file_source:
    os.mkdir(folderFrameOutput+'/'+inputVideo[:-4])
    cap = cv2.VideoCapture(inputVideo)
     # let's see if the video requires rotation
    rotateCode = check_rotation(inputVideo)
    #remove the path from the frame name
    vid=inputVideo[:-4].replace('C:\\Users\\Valerio\\DeepLearning\\First_Course_Env\\Dlib\\DlibProg\\dlib', '')
   
    i= 0
     #  loop frame by frame
    print("extraction in progress...")
    while True:
        grabbed, frame = cap.read()

         # if the grabbed parameter is false the video is finished
        if not grabbed:
            break

         # check if the video should be rotated
        if rotateCode is not None:
            frame = correct_rotation(frame, rotateCode)
        cv2.imwrite(folderFrameOutput+'/'+inputVideo[:-4]+'/'+"{0:0=4d}".format(i)+'.jpg',frame)
        i+=1
    cap.release()
    cv2.destroyAllWindows()
    print("extraction ended")
    folderFrame = 'prova3'
    images = load_images_from_folder(folderFrame+'/'+inputVideo[:-4]) #load_images_from_folder method call
    print('array population finished')
    i = 0
    print('face detection in progress...')
    for image in images:
        detect_face(image, i, vid) #recall method detect face for face detection
        i+=1
    print('face detection finished')